# Create patch files from cropped portions


In [1]:
import time
import os
import glob
from PIL import Image
Image.MAX_IMAGE_PIXELS = None
import numpy as np
import json
import tensorflow as tf
from tensorflow import keras
import keras.layers as kl
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
DIR_IMAGES_RAW = "/home/jrm/Martinez/images/Crop/"
DIR_IMAGES_TEST = "/home/jrm/Martinez/images/Crop/test/"
PATCH_SIZE=224  # matches VGG
IMAGE_SIZE = (PATCH_SIZE,PATCH_SIZE)
DF_Ypos = ['B7.','B15','D1.','D5.','E7.','E9.','F9.','G3.','H13','I1.','I5.','I13']
DF_Yneg = ['A3.','A5.','B13','C1.','C11','D3.','E5.','F3.','F7','F11','F13','F15','G15','H1.','H3.','H7.','H15']

In [3]:
FILENAMES_IMAGES_RAW = ['F15.DF1.135.Yneg.blue.1.png','F9.DF1.80.crop.png']

In [4]:
class patch_maker:
    def __init__(self):
        self.path=""
        self.w=0
        self.h=0
        self.im_width = 0
        self.im_height = 0
        self.patch_size = 10 # scalar, assumed square for now
        self.pixel_array = None
        self.output_dir = ""
        self.output_format='.png'
    def set_input_path(self,path):
        self.path=path
    def set_output_path(self,path):
        self.output_dir = path
    def set_patch_size(self,scalar):
        self.patch_size = scalar
    def load_pixel_array(self,filename,verbose=False):
        self.w=0
        self.h=0
        im = Image.open(self.path+filename)
        ima = np.array(im)   # convert to numpy
        self.im_width = ima.shape[0]
        self.im_height = ima.shape[1]    
        if verbose:
            print(filename, ima.size, ima.shape, self.patch_size)
        self.pixel_array = ima
        return ima
    def get_next_patch(self): 
        IM_WIDTH = self.im_width
        IM_HEIGHT = self.im_height
        PIXEL_ARRAY = self.pixel_array
        patch = None
        if self.w+self.patch_size > IM_WIDTH:
            self.h += self.patch_size
            self.w = 0
        if self.w+self.patch_size <= IM_WIDTH and self.h+self.patch_size <= IM_HEIGHT:
            patch = PIXEL_ARRAY[self.w:self.w+self.patch_size, self.h:self.h+self.patch_size]
            self.w += self.patch_size
        return patch
    def save_patch (self, prefix, pnum, patch):
        ext = self.output_format
        path= self.output_dir+prefix+'.'+str(pnum)+ext
        im =  Image.fromarray(patch)
        im.save(path)

In [5]:
pm = patch_maker()
pm.set_input_path(DIR_IMAGES_RAW)
pm.set_patch_size(PATCH_SIZE)
pm.set_output_path(DIR_IMAGES_TEST) 
total_counts = 0;
for filename in FILENAMES_IMAGES_RAW:
    prefix = filename[0:3]
    DIRECTORY = DIR_IMAGES_TEST
    if prefix in DF_Yneg:
        DIRECTORY += "blue/"
    elif prefix in DF_Ypos:
        DIRECTORY += "green/"
    else:
        DIRECTORY += "problem/"
    pixels = pm.load_pixel_array(filename,True)
    patch = pm.get_next_patch()
    file_counts = 0
    while patch is not None:
        file_counts += 1
        pm.set_output_path(DIR_IMAGES_TEST)
        pm.save_patch(filename,file_counts,patch)
        patch = pm.get_next_patch()
    print(file_counts, "patches")
    total_counts += file_counts
print('Total',total_counts,"patches")

F15.DF1.135.Yneg.blue.1.png 7426323 (1647, 1503, 3) 224
42 patches
F9.DF1.80.crop.png 12582846 (2094, 2003, 3) 224
72 patches
Total 114 patches
